In [62]:
import pandas as pd
import seaborn as sns

import pickle
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge

from sklearn.metrics import mean_squared_error

In [2]:
january_data_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet"
february_data_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet"

In [ ]:
def read_and_process_dataframe(url: str) -> pd.DataFrame:
    dataframe = pd.read_parquet(url)
    dataframe.lpep_dropoff_datetime = pd.to_datetime(dataframe.lpep_dropoff_datetime)
    dataframe.lpep_pickup_datetime = pd.to_datetime(dataframe.lpep_pickup_datetime)
    
    df = dataframe.copy()
    df["duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical_vars = ["PULocationID", "DOLocationID"]
    numerical_vars = ["trip_distance"]

    df[categorical_vars] = df[categorical_vars].astype(str)

    return df


In [41]:
df_train = read_and_process_dataframe(january_data_url)
df_val = read_and_process_dataframe(february_data_url)

In [42]:
len(df_train), len(df_val)

(73908, 61921)

In [55]:
df_train["PU_DO"] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]
df_val["PU_DO"] = df_val["PULocationID"] + "_" + df_val["DOLocationID"]

In [56]:
categorical_vars = ["PU_DO"] #["PULocationID", "DOLocationID"]
numerical_vars = ["trip_distance"]

In [57]:
dv = DictVectorizer()

train_dicts = df_train[categorical_vars + numerical_vars].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical_vars + numerical_vars].to_dict(orient="records")
X_val = dv.transform(val_dicts)

In [58]:
target = "duration"
y_train = df_train[target].values
y_val = df_val[target].values

In [59]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_val)
mse = mean_squared_error(y_val, y_pred, squared=False)
mse

7.479562160810692

In [63]:
with open("models/lin_reg.bin", "wb") as f_out:
    pickle.dump((dv, lr), f_out)

In [60]:
lasso = Lasso(alpha=0.001)
lasso.fit(X_train, y_train)

y_pred = lasso.predict(X_val)
mse = mean_squared_error(y_val, y_pred, squared=False)
mse

9.233436225720547

In [61]:
ridge = Ridge(alpha=0.001)
ridge.fit(X_train, y_train)

y_pred = ridge.predict(X_val)

mse = mean_squared_error(y_val, y_pred, squared=False)
mse

11.342565604878798